In [6]:
from pymongo import MongoClient
import pandas as pd
import pickle
client = MongoClient()
db = client.data_scientist_profiles

In [7]:
cursor = db.final_profile_details.find({}, {"assignments": 0, "dev_last_worked_ts": 0,
         "dev_portrait": 0, "dev_portrait_100": 0, "dev_portrait_32": 0, "dev_portrait_50": 0,
          "dev_timezone": 0, "permalink": 0, "ag_cny_recno":0, "ag_country":0, "ag_country_tz":0,
           "ag_description": 0, "ag_logo": 0, "ag_name": 0, "ag_recent_hours": 0, 
           "ag_total_hours": 0})

In [8]:
all_profiles = pd.DataFrame(list(cursor))
# drop duplicates
ap = all_profiles.drop_duplicates(subset = "ciphertext")
# remove new line character 
ap["dev_blurb"] = ap["dev_blurb"].apply(lambda x: x.replace("\n", ""))

In [10]:
length_profiles = ap["dev_blurb"].apply(lambda x: len(x))
len(length_profiles[length_profiles > 200])

73576

So most people have a profile of a decent length

In [11]:
with open("jobs.pkl", 'r') as picklefile: 
    jobs = pickle.load(picklefile)

In [12]:
# return all jobs that meet a certain criteria
def return_jobs(job_df, duration = "No Preference", job_type = "No Preference", 
               workload = "No Preference", payment_verification_status = "No Preference"):
    # get names of all arguments except for job_df
    args = locals().keys()
    args.remove("job_df")
    valid_jobs = job_df
    # for every argument entered, limit the dataframe
    for restriction in args:
        if locals()[restriction] != "No Preference":
            valid_jobs = valid_jobs[(valid_jobs[restriction] == locals()[restriction])]
    return valid_jobs

### Get cosine similarities of jobs

In [13]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, 
                                stop_words='english')
tf = tf_vectorizer.fit_transform(jobs["snippet"])
similarities = cosine_similarity(tf)
np.fill_diagonal(similarities, 0)

In [171]:
similarities[1].argmax()

473

In [172]:
similarities[1][473]

0.35361402919942991

In [174]:
# so 1 and 473 look similar, let's check them out
jobs["snippet"].iloc[1]

u'Looking for someone to do data collection.First project:This will start at the website http://www.signscompanies.comYou will then click on Browse by State, going to each website and collecting as much info as possible  clicking on each web link and collecting from there site Name, Address, Phone and Email address. Putting all that data in .cvs file.First state I need collected is Tennessee. Send sample of 5 shops to get job.Looking for a long time contract for the right person.'

In [175]:
jobs["snippet"].iloc[473]len()

u"Looking for Data miner to extract information from website and provide information in organize excel spreadsheet. Website might be membership only...not sure. I have a few different websites...I would like the following data mined from these website organization...-Narrow to Southern California-Contact Name ( Usually the CEO)-Their address-their email address-website address (www.website.com) Provide full URL-when and how long they've been a member There are a few websites. Please provide quote and what kind of information you can scrap. Please message for the website(s) if you would like to test to see if you can pull it off."

### Get cosine similarity of job titles

In [193]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, 
                                   stop_words='english')
tf_title = tfidf_vectorizer.fit_transform(jobs["title"])
from sklearn.metrics.pairwise import cosine_similarity
similarities_title = cosine_similarity(tf_title)
np.fill_diagonal(similarities_title, 0)

In [201]:
similarities_title[50].argmax()

2853

In [202]:
similarities_title[50][2853]

0.82592272673422917

In [203]:
jobs["title"].iloc[50]

u'SAS Programmer Needed'

In [204]:
jobs["title"].iloc[2853]

u'Need a SAS programmer'

Seems like it's working pretty well.

In [23]:
with open("highest_degree.pkl", 'r') as picklefile: 
    highest_degree = pickle.load(picklefile)
len(highest_degree)

79860

In [22]:
cursor = db.final_profile_details.find({}, {"dev_blurb" : 1, "ciphertext": 1}) 
just_blurbs = pd.DataFrame(list(cursor))
# drop duplicates
just_blurbs = just_blurbs.drop_duplicates(subset = "ciphertext")

In [24]:
just_blurbs["profile_length"] = just_blurbs["dev_blurb"].apply(lambda x: len(x))
just_blurbs["dev_blurb"] = just_blurbs["dev_blurb"].apply(lambda x: x.replace("\n", ""))
just_blurbs_long = just_blurbs[just_blurbs["profile_length"] > 250]

### Get cosine similarity of a profile and all jobs

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
def cosine_similarities(profile_text, jobs):
    # get jobs and profile into tfidf
    s1 = pd.Series(profile_text)
    jobs_and_blurb = s1.append(jobs["snippet"])
    tfidf_vectorizer_blurb = TfidfVectorizer(max_df=0.95, min_df=2, 
                                stop_words='english')
    tfidf_blurb =tfidf_vectorizer_blurb.fit_transform(jobs_and_blurb)
    profile_tfidf = tfidf_blurb[0]
    job_similarities = []
    for job in tfidf_blurb[1:]:
        job_similarities.append(cosine_similarity(profile_tfidf, job))
    return job_similarities

### Let's try computing a profile similarity to a job

In [81]:
s1 = pd.Series(just_blurbs_long["dev_blurb"].iloc[100])
jobs["snippet"] = jobs["snippet"].apply(lambda x: x.replace("\n", ""))
jobs["snippet"] = jobs["snippet"].apply(lambda x: x.replace("\u", " "))
jobs["snippet"] = jobs["snippet"].apply(lambda x: x.replace("\t", " "))
jobs["length_jobs"] = jobs["snippet"].apply(lambda x: len(x))
long_jobs = jobs[jobs["length_jobs"] > 150]
long_jobs_and_one_blurb = s1.append(long_jobs["snippet"])

In [82]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vectorizer_blurb = TfidfVectorizer(max_df=0.95, min_df=2, 
                                stop_words='english')
tfidf_blurb = tfidf_vectorizer_blurb.fit_transform(long_jobs_and_one_blurb)
from sklearn.metrics.pairwise import cosine_similarity
similarities_blurb_tfidf = cosine_similarity(tfidf_blurb)
np.fill_diagonal(similarities_blurb_tfidf, 0)

In [83]:
similarities_blurb_tfidf[0].argmax()

537

In [84]:
similarities_blurb[0][537]

0.22953526481905137

### Matching skills

In [16]:
def skill_match(persons_skills, job_skills = []):
    if job_skills == []:
        return .2
    per_skill_have = float(len([val for val in persons_skills if val in job_skills]))/len(job_skills)
    return per_skill_have

In [17]:
skill_match(["c+", "java"], ["java", "happy", "days"])

0.3333333333333333

## Ultimate Algorithm 

In [18]:
def norm_column(df, col_name):
    df["demeaned_col"] = df[col_name].apply(lambda x: x - df[col_name].mean())
    range_col = (df[col_name].max() - df[col_name].min())
    normed_col = df["demeaned_col"].apply(lambda x: x/range_col)
    return normed_col

In [30]:
def ultimate_matching_algorithm(jobs, profile_text, profile_skills = [], duration = "No Preference", job_type = "No Preference", 
workload = "No Preference", payment_verification_status = "No Preference"):
    # select only jobs meeting strict criteria
    select_jobs = return_jobs(jobs, duration, job_type, workload, payment_verification_status)
    # look for jobs with at least 150 words
    select_jobs["snippet_length"] = select_jobs["snippet"].apply(lambda x: len(x))
    select_jobs = select_jobs[select_jobs["snippet_length"] > 150]
    # add skill match column 
    select_jobs["skill_match"] = select_jobs["skills"].apply(lambda x: skill_match(profile_skills, x))
    # add profile text match column 
    select_jobs["similarity_score"] = cosine_similarities(profile_text, select_jobs)
    # add normalized columns
    select_jobs["norm_similarity"] = norm_column(select_jobs, "similarity_score")
    select_jobs["norm_skills"] = norm_column(select_jobs, "skill_match")
    #column with combination of profile and skill match, having standardized them
    select_jobs["skill_and_similarity"] = select_jobs["norm_similarity"] + .5*select_jobs["norm_skills"]
    # drop extra columns
    select_jobs = select_jobs.drop(["profile_length", "skill_match", "similarity_score", "demeaned_col", "norm_similarity", "norm_skills"], axis = 1)
    # return first 20 jobs
    return select_jobs.sort_values("skill_and_similarity", ascending = False).head(20)

In [31]:
ex_blurb = just_blurbs_long["dev_blurb"].iloc[100]
ultimate_matching_algorithm(jobs, ex_blurb, profile_skills = ["java", "c++"])

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-co

,budget,duration,id,job_type,skills,snippet,title,workload,country,feedback,jobs_posted,past_hires,payment_verification_status,reviews_count,snippet_length,skill_and_similarity
660,0.0,1 to 3 months,~011d5dfeb8b13de03e,Hourly,"[c++, deep-learning, machine-learning, python,...",We need a Deep Learning Research Engineer who ...,Deep Learning Research Engineer,30+ hrs/week,United Kingdom,5.000000,11,4,VERIFIED,2,1571,[[0.959040781435]]
2395,100.0,None,~0110ffd500c79812a5,Fixed,[],Looking for a developer with machine learning ...,Machine Learning on Images,30+ hrs/week,United States,0.000000,2,1,VERIFIED,0,222,[[0.890411138648]]
2573,0.0,3 to 6 months,~0177540d314299aa38,Hourly,[],2+ years of experience with Big Data technolog...,Graph Theory Expert,Less than 10 hrs/week,Israel,5.000000,21,15,VERIFIED,7,337,[[0.788956111668]]
2281,500.0,None,~01a945fe2e0b69af48,Fixed,[],We are an online eCommerce store. We need some...,Azure Machine Learning Specialist*​,30+ hrs/week,Barbados,3.959637,196,123,VERIFIED,64,459,[[0.753659023392]]
659,0.0,1 to 3 months,~01f5a59f6888037b60,Hourly,"[c++, deep-learning, machine-learning, python-...",RESPONSIBILITIESImplement newest deep learning...,Deep Learning Intern,30+ hrs/week,United Kingdom,5.000000,11,4,VERIFIED,2,1044,[[0.747069648267]]
1240,2000.0,None,~01b06be2766a469808,Fixed,"[accounting, blockchain, process-architect]",Need someone with in-depth knowledge of Blockc...,Analyse différent public blockchain technologi...,30+ hrs/week,Turks and Caicos Islands,4.997403,112,72,VERIFIED,41,286,[[0.727636893021]]
2666,0.0,Less than 1 month,~01d7c25065dbcd4d6e,Hourly,"[artificial-neural-networks, cuda, machine-lea...",Looking for an experienced deep learning devel...,tomato leaf disease classification problem usi...,Less than 10 hrs/week,South Korea,0.000000,1,0,VERIFIED,0,223,[[0.698875009122]]
2030,0.0,3 to 6 months,~01a538d78ee8f56158,Hourly,"[data-analysis, data-science]",We seek a Data Scientist at our Boston-based p...,Data Scientist - Contractor,30+ hrs/week,United States,0.000000,1,0,UNKNOWN,0,1645,[[0.687758297939]]
1261,5.0,None,~01ec666c5c918989f3,Fixed,"[artificial-intelligence, machine-learning, pr...","Looking for a programmer with a deep ""applicat...",Financial application developer (architecture ...,30+ hrs/week,United States,0.000000,1,0,UNKNOWN,0,628,[[0.655735334578]]
2451,1000.0,None,~015f8de2b325ec10ac,Fixed,[],BackgroundThere is currently a great deal of a...,Evaluation project to create an example image ...,30+ hrs/week,United States,0.000000,2,2,VERIFIED,0,874,[[0.650511847954]]
